<picture>
<a href="https://sambanova.ai/"\>
<source media="(prefers-color-scheme: dark)" srcset="../../images/SambaNova-light-logo-1.png" height="60">
<img alt="SambaNova logo" src="../../images/SambaNova-dark-logo-1.png" height="60">
</picture>
</a>

# Travel Planning 

Adapted from the same [Autogen example](https://microsoft.github.io/autogen/dev//user-guide/agentchat-user-guide/examples/travel-planning.html).

In this example, we'll walk through the process of creating a sophisticated travel planning system using [SambaNova Cloud](https://cloud.sambanova.ai/apis) and [Autogen AgentChats](https://microsoft.github.io/autogen/dev//user-guide/agentchat-user-guide/index.html). Our travel planner will utilize multiple AI agents, each with a specific role, to collaboratively create a comprehensive travel itinerary.  

First, let us import the necessary modules. Please, create a `.env` file and include `SAMBANOVA_URL` and `SAMBANOVA_API_KEY` after creating your [SambaNova Cloud API key](https://cloud.sambanova.ai/apis).

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
import getpass
load_dotenv('../../.env')

import os
api_url = os.getenv("SAMBANOVA_URL") # It has to be like this `https://api.sambanova.ai/v1`
api_key = os.getenv("SAMBANOVA_API_KEY")

if not api_key:
    api_key = getpass.getpass("insert your SambaNova API Key") 
    
if not api_url:
    api_url = "https://api.sambanova.ai/v1"

### Defining Agents 

In the next section we will define the agents that will be used in the travel planning team.

In [3]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=OpenAIChatCompletionClient(
        model="Meta-Llama-3.1-70B-Instruct", 
        base_url=api_url, 
        api_key=api_key, 
        model_info={
            'json_output': False, 
            'function_calling': True, 
            'family': 'unknown', 
            'vision': False
    }),
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=OpenAIChatCompletionClient(
        model="Meta-Llama-3.1-70B-Instruct", 
        base_url=api_url, 
        api_key=api_key, 
        model_info={
            'json_output': False, 
            'function_calling': True, 
            'family': 'unknown', 
            'vision': False
    }),
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=OpenAIChatCompletionClient(
        model="Meta-Llama-3.1-70B-Instruct", 
        base_url=api_url, 
        api_key=api_key, 
        model_info={
            'json_output': False, 
            'function_calling': True, 
            'family': 'unknown', 
            'vision': False
    }),
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=OpenAIChatCompletionClient(
        model="Meta-Llama-3.1-70B-Instruct", 
        base_url=api_url, 
        api_key=api_key, 
        model_info={
            'json_output': False, 
            'function_calling': True, 
            'family': 'unknown', 
            'vision': False
    }),
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

Finally, we create our agents team and specify the task.

In [ ]:
termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)
await Console(group_chat.run_stream(task="Plan a 3 day trip to Peru."))